In [1]:
from __future__ import print_function, division
import numpy as np
np.random.seed(42)
import tensorflow as tf
import pdb
import fxns

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# data locations
raw_data_path = '../data/shuffled_examples'
outbase = '../output/shuffled_examples'
train_out_path = '%s/texttrain' % outbase
valid_out_path = '%s/textvalidate' % outbase
test_out_path = '%s/texttest' % outbase

# training hyperparameters
batch_size = 50
num_batches_in_train = int(434786 / batch_size)
num_epochs = 2
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

# retrieving models
global_step_to_load = None

In [3]:
tf.reset_default_graph()

# get training data
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('inputs are of size', info['seq_len'])

# get validation data
(seqv, labelv), infov = fxns.get_seq_and_label(valid_out_path)
seqv_batch, labelv_batch = tf.train.shuffle_batch([seqv, labelv],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('outputs are of size', info['label_len'])

reload(fxns)
print(test_out_path)
(seqt, labelt), info = fxns.get_seq_and_label(test_out_path, num_epochs=None)
seqt_batch, labelt_batch = tf.train.shuffle_batch([seqt, labelt],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

inputs are of size 4000
outputs are of size 24
../output/shuffled_examples/texttest


/anaconda2/lib/python2.7/site-packages/pandas/core/series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [4]:
# # get model -- logistic
# import logreg_model
# modelname='logreg'
# X, Y, loss, logits = logreg_model.get_logreg_model(info['seq_len'], info['label_len'])

# get model -- generic convnet
import novel_model; reload(novel_model)
modelname='freeconv'
#conv_infos = [(7,(1,20),(2,2),4),(5,(1,20),(2,2),7)]#, (8,(1,20),(2,2),5)]
conv_infos = [(14,(1,21),(2,2),2,4),(14,(1,8),(2,2),2,14),(7,(1,5),(2,2),2,14)]#, (8,(1,20),(2,2),5)]
X, Y, loss, logits = novel_model.get_novel_model(info['seq_len'], info['label_len'], conv_infos)

500.0
250.0
125.0
4000 h 15 125.0


In [5]:
# define optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate,
                                                 name='SGD-Optimizer')
    update = optimizer.minimize(loss)

# define other summaries we want
with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram-loss', loss)
    summary_op = tf.summary.merge_all()

In [12]:
if tf.gfile.Exists('log/' + modelname):
   tf.gfile.DeleteRecursively('log/' + modelname) 

# train

print_every = 25
save_every = 250

with tf.Session() as sess:
    
    saver = tf.train.Saver(max_to_keep=None)
    
    if global_step_to_load is None:
        global_step = 0
    else:
        saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load))
        global_step = global_step_to_load
    
#    print(layer)
    
    writer = tf.summary.FileWriter('log/' + modelname + '/train', sess.graph)
    writerv = tf.summary.FileWriter('log/' + modelname + '/valid')
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    if global_step_to_load is None:
        sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        total_loss = 0
        for i in range(num_batches_in_train):
            batch_X, batch_Y = sess.run([seq_batch, label_batch])
            batch_Xv, batch_Yv = sess.run([seqv_batch, labelv_batch])
            _, _loss = sess.run([update, loss],
                            feed_dict={X: batch_X, Y: batch_Y})
            total_loss += _loss #/ num_batches_in_train
            
            summary = sess.run(summary_op, feed_dict={X: batch_X,
                                                    Y: batch_Y})
            summaryv = sess.run(summary_op, feed_dict={X: batch_Xv,
                                                    Y: batch_Yv})
#            writer.add_summary(summary, global_step=epoch*num_batches_in_train + i)
#            writerv.add_summary(summaryv, global_step=epoch*num_batches_in_train + i)
            writer.add_summary(summary, global_step=global_step)
            writerv.add_summary(summaryv, global_step=global_step)
            
            global_step += 1
            
            if global_step % save_every == 0:
                print('saving')
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
                saver.save(sess, 'log/%s' % modelname, global_step=global_step)
                
            if global_step % print_every == 0:
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
        
        print('saving')
        print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))

        saver.save(sess, 'log/%s' % modelname, global_step=global_step)
    
    writer.close()
    coord.request_stop()
    coord.join(threads)

Epoch: 0 global_step 25 i 24 Avg loss in epoch(incomplete): 0.3382432895898819
Epoch: 0 global_step 50 i 49 Avg loss in epoch(incomplete): 0.2567225942015648
Epoch: 0 global_step 75 i 74 Avg loss in epoch(incomplete): 0.2265290226538976
Epoch: 0 global_step 100 i 99 Avg loss in epoch(incomplete): 0.21146436870098115
Epoch: 0 global_step 125 i 124 Avg loss in epoch(incomplete): 0.2021866956949234
Epoch: 0 global_step 150 i 149 Avg loss in epoch(incomplete): 0.1958569437265396
Epoch: 0 global_step 175 i 174 Avg loss in epoch(incomplete): 0.1915256370816912
Epoch: 0 global_step 200 i 199 Avg loss in epoch(incomplete): 0.1881510356068611
Epoch: 0 global_step 225 i 224 Avg loss in epoch(incomplete): 0.18553672558731504
saving
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.18341794615983964
INFO:tensorflow:log/freeconv-250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.18341794615983964
Epoch: 0 globa

Epoch: 0 global_step 2075 i 2074 Avg loss in epoch(incomplete): 0.16733446628214366
Epoch: 0 global_step 2100 i 2099 Avg loss in epoch(incomplete): 0.16731365539488338
Epoch: 0 global_step 2125 i 2124 Avg loss in epoch(incomplete): 0.1672931893152349
Epoch: 0 global_step 2150 i 2149 Avg loss in epoch(incomplete): 0.16727765583021698
Epoch: 0 global_step 2175 i 2174 Avg loss in epoch(incomplete): 0.16725061557758814
Epoch: 0 global_step 2200 i 2199 Avg loss in epoch(incomplete): 0.16721881720830092
Epoch: 0 global_step 2225 i 2224 Avg loss in epoch(incomplete): 0.16719722680831223
saving
Epoch: 0 global_step 2250 i 2249 Avg loss in epoch(incomplete): 0.16717644445763694
INFO:tensorflow:log/freeconv-2250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 0 global_step 2250 i 2249 Avg loss in epoch(incomplete): 0.16717644445763694
Epoch: 0 global_step 2275 i 2274 Avg loss in epoch(incomplete): 0.1671511212679056
Epoch: 0 global_step 2300 i 2299 Avg loss in epoch(incomplete):

Epoch: 0 global_step 4100 i 4099 Avg loss in epoch(incomplete): 0.16617691774193832
Epoch: 0 global_step 4125 i 4124 Avg loss in epoch(incomplete): 0.16616858550635252
Epoch: 0 global_step 4150 i 4149 Avg loss in epoch(incomplete): 0.16616439175533962
Epoch: 0 global_step 4175 i 4174 Avg loss in epoch(incomplete): 0.1661597754427059
Epoch: 0 global_step 4200 i 4199 Avg loss in epoch(incomplete): 0.16615287560437406
Epoch: 0 global_step 4225 i 4224 Avg loss in epoch(incomplete): 0.16614311981130633
saving
Epoch: 0 global_step 4250 i 4249 Avg loss in epoch(incomplete): 0.1661372351506177
INFO:tensorflow:log/freeconv-4250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 0 global_step 4250 i 4249 Avg loss in epoch(incomplete): 0.1661372351506177
Epoch: 0 global_step 4275 i 4274 Avg loss in epoch(incomplete): 0.16613619099932106
Epoch: 0 global_step 4300 i 4299 Avg loss in epoch(incomplete): 0.1661376542372759
Epoch: 0 global_step 4325 i 4324 Avg loss in epoch(incomplete): 0

Epoch: 0 global_step 6125 i 6124 Avg loss in epoch(incomplete): 0.16581527894613693
Epoch: 0 global_step 6150 i 6149 Avg loss in epoch(incomplete): 0.1658097208224661
Epoch: 0 global_step 6175 i 6174 Avg loss in epoch(incomplete): 0.16580504441792182
Epoch: 0 global_step 6200 i 6199 Avg loss in epoch(incomplete): 0.16580132300574935
Epoch: 0 global_step 6225 i 6224 Avg loss in epoch(incomplete): 0.16580115174433313
saving
Epoch: 0 global_step 6250 i 6249 Avg loss in epoch(incomplete): 0.16579696593284607
INFO:tensorflow:log/freeconv-6250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 0 global_step 6250 i 6249 Avg loss in epoch(incomplete): 0.16579696593284607
Epoch: 0 global_step 6275 i 6274 Avg loss in epoch(incomplete): 0.1657947330242134
Epoch: 0 global_step 6300 i 6299 Avg loss in epoch(incomplete): 0.16578881272957438
Epoch: 0 global_step 6325 i 6324 Avg loss in epoch(incomplete): 0.16578863861296958
Epoch: 0 global_step 6350 i 6349 Avg loss in epoch(incomplete):

Epoch: 0 global_step 8150 i 8149 Avg loss in epoch(incomplete): 0.16564599517664294
Epoch: 0 global_step 8175 i 8174 Avg loss in epoch(incomplete): 0.16564383128185156
Epoch: 0 global_step 8200 i 8199 Avg loss in epoch(incomplete): 0.16564431323874287
Epoch: 0 global_step 8225 i 8224 Avg loss in epoch(incomplete): 0.16564280032386897
saving
Epoch: 0 global_step 8250 i 8249 Avg loss in epoch(incomplete): 0.1656410465981021
INFO:tensorflow:log/freeconv-8250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 0 global_step 8250 i 8249 Avg loss in epoch(incomplete): 0.1656410465981021
Epoch: 0 global_step 8275 i 8274 Avg loss in epoch(incomplete): 0.16563942029396933
Epoch: 0 global_step 8300 i 8299 Avg loss in epoch(incomplete): 0.16563467475664184
Epoch: 0 global_step 8325 i 8324 Avg loss in epoch(incomplete): 0.16563200795077704
Epoch: 0 global_step 8350 i 8349 Avg loss in epoch(incomplete): 0.16563058461436256
Epoch: 0 global_step 8375 i 8374 Avg loss in epoch(incomplete):

Epoch: 1 global_step 10125 i 1429 Avg loss in epoch(incomplete): 0.16514160981128265
Epoch: 1 global_step 10150 i 1454 Avg loss in epoch(incomplete): 0.16516016220923552
Epoch: 1 global_step 10175 i 1479 Avg loss in epoch(incomplete): 0.16515793189205027
Epoch: 1 global_step 10200 i 1504 Avg loss in epoch(incomplete): 0.16516145061812923
Epoch: 1 global_step 10225 i 1529 Avg loss in epoch(incomplete): 0.1651540249391319
saving
Epoch: 1 global_step 10250 i 1554 Avg loss in epoch(incomplete): 0.1651470369752197
INFO:tensorflow:log/freeconv-10250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 1 global_step 10250 i 1554 Avg loss in epoch(incomplete): 0.1651470369752197
Epoch: 1 global_step 10275 i 1579 Avg loss in epoch(incomplete): 0.1651409171710286
Epoch: 1 global_step 10300 i 1604 Avg loss in epoch(incomplete): 0.16513778447547806
Epoch: 1 global_step 10325 i 1629 Avg loss in epoch(incomplete): 0.1651285009790052
Epoch: 1 global_step 10350 i 1654 Avg loss in epoch(inc

Epoch: 1 global_step 12125 i 3429 Avg loss in epoch(incomplete): 0.16509385496464832
Epoch: 1 global_step 12150 i 3454 Avg loss in epoch(incomplete): 0.16509031370476254
Epoch: 1 global_step 12175 i 3479 Avg loss in epoch(incomplete): 0.1650944106195165
Epoch: 1 global_step 12200 i 3504 Avg loss in epoch(incomplete): 0.16508729267392452
Epoch: 1 global_step 12225 i 3529 Avg loss in epoch(incomplete): 0.16508732728512363
saving
Epoch: 1 global_step 12250 i 3554 Avg loss in epoch(incomplete): 0.1650824319503143
INFO:tensorflow:log/freeconv-12250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 1 global_step 12250 i 3554 Avg loss in epoch(incomplete): 0.1650824319503143
Epoch: 1 global_step 12275 i 3579 Avg loss in epoch(incomplete): 0.16508219168422608
Epoch: 1 global_step 12300 i 3604 Avg loss in epoch(incomplete): 0.16507761005555047
Epoch: 1 global_step 12325 i 3629 Avg loss in epoch(incomplete): 0.16507557853851107
Epoch: 1 global_step 12350 i 3654 Avg loss in epoch(i

Epoch: 1 global_step 14125 i 5429 Avg loss in epoch(incomplete): 0.16506507754545405
Epoch: 1 global_step 14150 i 5454 Avg loss in epoch(incomplete): 0.16506874180790704
Epoch: 1 global_step 14175 i 5479 Avg loss in epoch(incomplete): 0.16506936461222868
Epoch: 1 global_step 14200 i 5504 Avg loss in epoch(incomplete): 0.16507534031755378
Epoch: 1 global_step 14225 i 5529 Avg loss in epoch(incomplete): 0.16507405131839714
saving
Epoch: 1 global_step 14250 i 5554 Avg loss in epoch(incomplete): 0.16507130471250156
INFO:tensorflow:log/freeconv-14250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 1 global_step 14250 i 5554 Avg loss in epoch(incomplete): 0.16507130471250156
Epoch: 1 global_step 14275 i 5579 Avg loss in epoch(incomplete): 0.1650741394435633
Epoch: 1 global_step 14300 i 5604 Avg loss in epoch(incomplete): 0.16507595891829158
Epoch: 1 global_step 14325 i 5629 Avg loss in epoch(incomplete): 0.16507566946279092
Epoch: 1 global_step 14350 i 5654 Avg loss in epoch

Epoch: 1 global_step 16125 i 7429 Avg loss in epoch(incomplete): 0.16505170866425875
Epoch: 1 global_step 16150 i 7454 Avg loss in epoch(incomplete): 0.16505335056286063
Epoch: 1 global_step 16175 i 7479 Avg loss in epoch(incomplete): 0.16505140417360048
Epoch: 1 global_step 16200 i 7504 Avg loss in epoch(incomplete): 0.1650493938791045
Epoch: 1 global_step 16225 i 7529 Avg loss in epoch(incomplete): 0.16505064155317714
saving
Epoch: 1 global_step 16250 i 7554 Avg loss in epoch(incomplete): 0.1650543674225053
INFO:tensorflow:log/freeconv-16250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 1 global_step 16250 i 7554 Avg loss in epoch(incomplete): 0.1650543674225053
Epoch: 1 global_step 16275 i 7579 Avg loss in epoch(incomplete): 0.16505131938526688
Epoch: 1 global_step 16300 i 7604 Avg loss in epoch(incomplete): 0.16505084204524853
Epoch: 1 global_step 16325 i 7629 Avg loss in epoch(incomplete): 0.16504867124330952
Epoch: 1 global_step 16350 i 7654 Avg loss in epoch(i

In [8]:
#
# get test data

global_step_to_load_test = 17390


scores = []
labels = []

with tf.Session() as sess:
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=None)
    saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load_test))
    
    idx = 0
    while idx < 45458 / batch_size:
        batch_Xt, batch_Yt = sess.run([seqt_batch, labelt_batch])
        batch_logits = sess.run(logits, feed_dict={X: batch_Xt, Y: batch_Yt})
        scores.append(batch_logits)
        labels.append(batch_Yt)
        #print('asdf')
        idx += 1
        if idx % 100 == 0:
            print(idx)

scores_arr = np.concatenate(scores, axis=0)
labels_arr = np.concatenate(labels, axis=0)



INFO:tensorflow:Restoring parameters from log/freeconv-17390
100
200
300
400
500
600
700
800
900
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer_2/input_producer_2_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer_2, input_producer_2/RandomShuffle)]]
accuracies:
[0.92912088 0.98854945 0.99096703 0.99463736 0.97523077 0.97362637
 0.97507692 0.9076044  0.98134066 0.97830769 0.94145055 0.96782418
 0.94964835 0.968      0.90430769 0.9401978  0.94320879 0.98312088
 0.97918681 0.92120879 0.99       0.91685714 0.97134066 0.92918681]
aucs:
[0.44973939002195845, 0.7981133784804416, 0.66104237528815, 0.5024992610344938, 0.6601810124948881, 0.6678225921745673, 0.5143228293970603, 0.45235949837537803, 0.7149802872422502, 0.6651232836571472, 0.5944746586744556, 0.6555319927014509, 0.483

ValueError: Reordering is not turned on, and the x array is not increasing: [0.0709025  0.07088207 0.07088363 ... 0.         0.         1.        ]

In [97]:
accuracies = np.mean((scores_arr > 0).astype(int) == labels_arr.astype(int), axis=0)

print('accuracies:')
print(accuracies)

import sklearn.metrics
print('aucs:')
aucs = [sklearn.metrics.roc_auc_score(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(aucs)

import sklearn.metrics
def prec_recall(ys_true, ys_hat):
    ys, xs, thresholds = sklearn.metrics.precision_recall_curve(ys_true, ys_hat)
    return sklearn.metrics.auc(xs, ys, reorder=True)

print('auprcs:')
auprcs = [prec_recall(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(auprcs)

props = np.mean(labels_arr, axis=0)

import json
ids = sorted(json.loads(info['tf_to_pos'].replace("'", '"')).keys())

from IPython.display import display_pretty, display_html
import pandas as pd
display_html(pd.DataFrame({'props':props, 'acc':accuracies, 'auprcs':auprcs, 'aucs':aucs}, index=ids).sort_values(by='aucs').to_html(), raw=True)
results = pd.DataFrame({'props':props, 'acc':accuracies, 'auprcs':auprcs, 'aucs':aucs}, index=ids).sort_values(by='aucs')
display_html(results.to_html(), raw=True)
results.to_csv('stats.' + modelname + '.tsv', index_label='id', sep='\t')

accuracies:
[0.92912088 0.98854945 0.99096703 0.99463736 0.97523077 0.97362637
 0.97507692 0.9076044  0.98134066 0.97830769 0.94145055 0.96782418
 0.94964835 0.968      0.90430769 0.9401978  0.94320879 0.98312088
 0.97918681 0.92120879 0.99       0.91685714 0.97134066 0.92918681]
aucs:
[0.44973939002195845, 0.7981133784804416, 0.66104237528815, 0.5024992610344938, 0.6601810124948881, 0.6678225921745673, 0.5143228293970603, 0.45235949837537803, 0.7149802872422502, 0.6651232836571472, 0.5944746586744556, 0.6555319927014509, 0.48381148547235, 0.6365781914139806, 0.4014560946699744, 0.36607708056806987, 0.3369110227423894, 0.6381399309359072, 0.6359718907687298, 0.48225079946115296, 0.5827983249961273, 0.43690013264927596, 0.7815913926641336, 0.4705150060656418]
auprcs:
[0.0611933401763262, 0.03295243219534374, 0.015253773054116087, 0.007392082010746848, 0.04033729402174586, 0.04725718350150342, 0.026659818031494777, 0.08031653243220327, 0.040010976733484714, 0.036880772486206026, 0.084595

,acc,aucs,auprcs,props
SydhHuvecGata2Ucd,0.943209,0.336911,0.038821,0.056791
SydhHuvecCjun,0.940198,0.366077,0.042769,0.059802
SydhHuvecCfosUcd,0.904308,0.401456,0.073894,0.095692
UtaHuvecCtcf,0.916857,0.436900,0.069306,0.083143
BroadHuvecCtcf,0.929121,0.449739,0.061193,0.070879
HaibGm12891Pu1Pcr1x,0.907604,0.452359,0.080317,0.092396
UwHuvecCtcf,0.929187,0.470515,0.064081,0.070813
UtaGm12891Ctcf,0.921209,0.482251,0.075564,0.078791
SydhGm12891NfkbTnfaIggrab,0.949648,0.483811,0.046756,0.050352
HaibGm12891Pax5c20V0416101,0.994637,0.502499,0.007392,0.005363
